In [ ]:
"""CIFAR 10.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1avE06Z_UruJoRBtQHg6IU56A6xJQ67ZJ
"""

In [ ]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import datasets
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import Xception
from tensorflow.keras.applications import VGG19
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50
import tensorflow as tf
import numpy as np
from keras.optimizers import SGD,Adam
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [ ]:
#Divide the data in Train, Validation and Test Datasets
(X_train,y_train),(X_test,y_test)=datasets.cifar10.load_data()

Print the dimensions of the datasets

In [ ]:
print((X_train.shape,y_train.shape))
# print((x_val.shape,y_val.shape))
print((X_test.shape,y_test.shape))

In [ ]:
y_train = y_train.reshape(-1, )
y_train

In [ ]:
y_classes = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]
len(y_classes)

In [ ]:
# Normalizing pixel values to the range [0, 1
X_train = X_train / 255.0
X_test = X_test / 255.0
# X_train[0]

In [ ]:
# Define data augmentation transformations for both training and validation
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    # shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    # brightness_range=[0.5, 1.5],  # Adjust brightness
    # channel_shift_range=50.0,?

In [ ]:
)

In [ ]:
val_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    # shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',

In [ ]:
)

In [ ]:
train_generator = train_datagen.flow(
    X_train,
    y_train,
    batch_size = 32
)
val_generator = val_datagen.flow(
    X_test,
    y_test,
    batch_size = 32
)

In [ ]:
#Build the model
base_model = VGG19(include_top=False,
                     weights='imagenet',
                     input_shape=(32,32,3),
                     )
# base_model.

In [ ]:
from typing import Optional
model= Sequential()
model.add(base_model)
model.add(Flatten())

In [ ]:
model.add(Dense(1024, activation='relu', input_dim=512,))

In [ ]:
model.add(Dense(512,activation=('relu')))

In [ ]:
model.add(Dense(256,activation=('relu')))

In [ ]:
model.add(Dense(128,activation=('relu'),))
model.add(Dropout(0.3))
model.add(Dense(10,activation=('softmax')))

In [ ]:
learn_rate=.001

In [ ]:
sgd=SGD(learning_rate=learn_rate,momentum=.9,nesterov=False)
adam=Adam(learning_rate=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None, amsgrad=False)

In [ ]:
model.compile(optimizer=sgd,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
batch_size = 32
history = model.fit(
    train_generator,
    steps_per_epoch = len(X_train)// batch_size,
    validation_data = val_generator,
    validation_steps = len(X_test)//batch_size,
    epochs = 10
    )

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
model.save("/content/drive/MyDrive/Colab Notebooks.h5")

In [ ]:
import matplotlib.pyplot as plt
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
## Load and preprocess an image for inference
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import load_model

In [ ]:
# Load a pre-trained model (e.g., your custom model)
model = load_model("/content/drive/MyDrive/Colab Notebooks.h5")

In [ ]:
# Load and preprocess an image
img_path = '/content/istockphoto-1314888871-612x612.jpg'
img = image.load_img(img_path, target_size=(32, 32))
x = image.img_to_array(img)
x = preprocess_input(x)
x = np.expand_dims(x, axis=0)

In [ ]:
# Make predictions
predictions = model.predict(x)
class_index = np.argmax(predictions)

In [ ]:
print(f'Predicted class index: {class_index}')

In [ ]:
import numpy as np
import cv2
from keras.preprocessing import image
from keras.models import load_model
model= load_model('/content/drive/MyDrive/Colab Notebooks.h5')
test_img = cv2.imread('/content/horse.jpg')
img = cv2.resize(test_img, (32, 32), interpolation = cv2.INTER_CUBIC)
img = np.reshape(img, (1, 32, 32, 3))
img = img/255.
pred = model.predict(img)
print(pred)
class_num = np.argmax(pred)
print(class_num)
# pred = {0:"airplane",1:"Dog"}
pred = {0:"airplane",1:"automobile",2:"bird",3:"cat",4:"deer",5:"dog",6:"frog",7:"horse",8:"ship",9:"truck"}
print("The predicted class = ",pred[class_num])
# y_classes = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]